In [45]:
from bs4 import BeautifulSoup

In [91]:
import requests
import pandas as pd
import random
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
    client_id="aac1ac9a179f4d02b944ef2bd3ec22ee",
    client_secret="6c4b68aad01b4201affc7d7a2d4cbe11"))

In [47]:
def create_soup(html):
    url = html
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    return soup
soup = create_soup("https://www.billboard.com/charts/hot-100")

In [48]:
def create_list(part_html, soup):
    list_name = []
    for element in soup.select(part_html):
        list_name.append(element.get_text())
    return list_name
songs = create_list(".chart-element__information__song", soup)
artists = create_list(".chart-element__information__artist", soup)
rankLW = create_list("span.chart-element__meta.text--center.color--secondary.text--last", soup)
peaks = create_list("span.chart-element__meta.text--center.color--secondary.text--peak", soup)
wocs = create_list("span.chart-element__meta.text--center.color--secondary.text--week", soup)

In [49]:
dftop100 = pd.DataFrame({
    "Song": songs,
    "Artist": artists,
    "Rank Last Week": rankLW,
    "Peak": peaks,
    "Weeks on Chart": wocs})

In [51]:
low_top100 = []

for song in songs:
    low_top100.append(song.lower())

In [101]:
input_song = str(input("Choose a song: ")).lower()
if input_song not in low_top100:
    
#search Spotify
    not_100 = spotify.search(q=input_song, limit = 1)
#get features + add to dataframe
    features = spotify.audio_features(not_100["tracks"]["items"][0]["uri"])
    features_df = pd.DataFrame(features)
    features_n = features_df.drop(["type","id","uri","track_href","analysis_url"], axis=1)
#scale dataframe
    scaled_df = pd.DataFrame(StandardScaler().fit_transform(features_n))
    scale = [list(scaled_df.loc[0])]
    
    
    

Choose a song: umbrella


In [43]:
def choose_song():
    input_song = str(input("Choose a song: ")).lower()
    if input_song not in low_top100:
        print("Feature coming soon..")
    else:
        random_song = random.sample(songs, k = 1)[0]
        print("If you liked that, you might also like:" + random_song)
        input_like_song = str(input("Did we do okay? (Yes/No) ").lower())
        while input_like_song == "no":
            print("Sorry about that, feature coming soon..")
        if input_like_song == "yes":
            print("Glad we could help! We'll keep 'em coming: ", random.sample(songs, k = 1)[0])
            input_like_song = str(input().lower())
    return random_song


In [44]:
choose_song()

KeyboardInterrupt: Interrupted by user